In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

print(os.getcwd())

In [ ]:
# Define speeds for 0-40 Hz
speeds = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))



In [ ]:
class CrossWireDataset(Dataset):
    def __init__(self, magFile, angFile, readingsFile, transform=None, target_transform=None):
        
        # Construct the labels
        tmpMag = pd.read_csv(magFile)
        tmpAng = pd.read_csv(angFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.angs = torch.Tensor(tmpAng.to_numpy())
        
        # Construct the features and place them into readings (X). 
        tmpReadings = pd.read_csv(readingsFile)
        tmpReadings = tmpReadings.to_numpy()
        print(tmpReadings.shape)
        LL = tmpReadings.shape[0]
        tmpReadings2 = np.zeros((LL, 3))
        for k in range(LL):
            tmpReadings2[k, 0] = np.max(np.abs(tmpReadings[k, :]))
            tmpReadings2[k, 1] = np.argmax(np.abs(tmpReadings[k, :]))
            tt = int(tmpReadings2[k,1])
            tmpReadings2[k, 2] = np.abs(tmpReadings[k, (tt-1)%6])/(np.abs(tmpReadings[k, (tt+1)%6]) + 0.05)
        self.readings = torch.Tensor(tmpReadings2)
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        ang = self.angs[idx]
        label = torch.cat((mag, ang), 0)
        
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            mag = self.target_transform(mag)

        return reading, label

In [ ]:
class WindMagDataset(Dataset):
    def __init__(self, magFile, readingsFile, transform=None, target_transform=None):
        tmpMag = pd.read_csv(magFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            mag = self.target_transform(mag)

        return reading, mag

class WindAngDataset(Dataset):
    def __init__(self, angFile, readingsFile, transform=None, target_transform=None):
        tmpAng = pd.read_csv(angFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.angs = torch.Tensor(tmpAng.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.angs)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        ang = self.angs[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            ang = self.target_transform(ang)

        return reading, ang

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, crosswire=False, fullAngles=False):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        
        if crosswire:
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(3, 25),
                nn.ReLU(),
                nn.Linear(25, 15),
                nn.ReLU(),
                nn.Linear(15, 2),
            )
        else:
            if fullAngles:
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(6, 20),
                    nn.ReLU(),
                    nn.Linear(20, 8),
                    nn.ReLU(),
                    nn.Linear(8, 1),
                )
            else:
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(6, 10),
                    nn.ReLU(),
                    nn.Linear(10, 5),
                    nn.ReLU(),
                    nn.Linear(5, 1),
                )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

In [ ]:
# Define miscellaneous learning quantities
learning_rate = 1e-3
batch_size = 64
epochs = 8
loss_fn = nn.MSELoss()


In [ ]:
# Make CrossWire Dataset
training_data3 = CrossWireDataset('CrossTrain/crossmags.csv', 'CrossTrain/crossangsrad.csv', 'CrossTrain/crossreadings.csv', transform=None)
train_dataloader3 = DataLoader(training_data3, batch_size=64, shuffle=True)
testing_data3 = CrossWireDataset('CrossTest/crossmags.csv', 'CrossTest/crossangsrad.csv', 'CrossTest/crossreadings.csv', transform=None)
test_dataloader3 = DataLoader(testing_data3, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

crossModel = NeuralNetwork(crosswire=True)

optimizerCross = torch.optim.Adam(crossModel.parameters(), lr=learning_rate)

X = torch.rand(1, 3)
logits = crossModel(X)
print(logits[0])

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader3, crossModel, loss_fn, optimizerCross)
    test_loop(test_dataloader3, crossModel, loss_fn)
print("Done!")

In [ ]:
# Make fullMag Data
training_data4 = WindMagDataset('CrossTrain/crossmags.csv', 'CrossTrain/crossreadings.csv', transform=None)
train_dataloader4 = DataLoader(training_data4, batch_size=64, shuffle=True)
testing_data4 = WindMagDataset('CrossTest/crossmags.csv', 'CrossTest/crossreadings.csv', transform=None)
test_dataloader4 = DataLoader(testing_data4, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

fullMagModel = NeuralNetwork()

optimizerFullMag = torch.optim.Adam(fullMagModel.parameters(), lr=learning_rate)

X = torch.rand(1, 6)
logits = fullMagModel(X)
print(logits[0])

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader4, fullMagModel, loss_fn, optimizerFullMag)
    test_loop(test_dataloader4, fullMagModel, loss_fn)
print("Done!")

In [ ]:
# Make fullAng Data
training_data5 = WindAngDataset('CrossTrain/crossangsrad.csv', 'CrossTrain/crossreadings.csv', transform=None)
train_dataloader5 = DataLoader(training_data5, batch_size=64, shuffle=True)
testing_data5 = WindAngDataset('CrossTest/crossangsrad.csv', 'CrossTest/crossreadings.csv', transform=None)
test_dataloader5 = DataLoader(testing_data5, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

fullAngModel = NeuralNetwork(fullAngles=True)

optimizerFullAng = torch.optim.Adam(fullAngModel.parameters(), lr=learning_rate)

X = torch.rand(1, 6)
logits = fullAngModel(X)
print(logits[0])

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader5, fullAngModel, loss_fn, optimizerFullAng)
    test_loop(test_dataloader5, fullAngModel, loss_fn)
print("Done!")

In [ ]:
# Make Wind Mag Data
training_data = WindMagDataset('MagTrain/mags.csv', 'MagTrain/readings.csv', transform=None)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
testing_data = WindMagDataset('MagTest/mags.csv', 'MagTest/readings.csv', transform=None)
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

magModel = NeuralNetwork()

optimizerMag = torch.optim.Adam(magModel.parameters(), lr=learning_rate)

X = torch.rand(1, 6)
logits = magModel(X)
print(logits[0])

In [ ]:
epochs = 7
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, magModel, loss_fn, optimizerMag)
    test_loop(test_dataloader, magModel, loss_fn)
print("Done!")

In [ ]:
# Make Wind Mag Data
training_data2 = WindAngDataset('MagTrain/angsrad.csv', 'MagTrain/readings.csv', transform=None)
train_dataloader2 = DataLoader(training_data2, batch_size=64, shuffle=True)
testing_data2 = WindAngDataset('MagTest/angsrad.csv', 'MagTest/readings.csv', transform=None)
test_dataloader2 = DataLoader(testing_data2, batch_size=64, shuffle=True)

angModel = NeuralNetwork()
optimizerAng = torch.optim.Adam(angModel.parameters(), lr=learning_rate)
X = torch.rand(1, 6)
logits = angModel(X)
print(logits[0])

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader2, angModel, loss_fn, optimizerAng)
    test_loop(test_dataloader2, angModel, loss_fn)
print("Done!")